<h1>Generate folders + Convert wav to spectrograms for the cnn model</h1>

In [1]:
"""
    Convert wavs from [machine]/wavs to color spectrograms into folders
    # construct dataset of images spectrogram in folders:
        # _data_png/[machine]/png_train/normal
        # _data_png/[machine]/png_train/anormal
        # _data_png/[machine]/png_test is used for images tests never seen by the model

"""

from os import listdir
from os.path import isfile, join
import numpy as np
import sys

from SoundFile import SoundFile
import utils 
import datetime
import os
import shutil
from pathlib import Path


In [2]:
nbImagesTotestEachClass = 100 # put this count of images in png_test (for each class)
start_time = datetime.datetime.now()

# create folders in order to export pngs files (remove all previous png) -> _data_png/[machine]/normal _data_png/[machine]/anomaly
mode = 0o777
cwd = os.getcwd() 
path = Path(cwd)

png_root_folder = str(path.parent.absolute()) + '/_data_png'
model_root_folder = str(path.parent.absolute()) + '/_classifiers'

# delete old folders if needed
try:
    shutil.rmtree(png_root_folder)
    shutil.rmtree(model_root_folder)
except OSError as e:
    print ("Error: %s - %s." % (e.filename, e.strerror))

os.mkdir(os.path.join(png_root_folder), mode)
os.mkdir(os.path.join(model_root_folder), mode)

for folder_machine in utils.list_datasets: # ['valve']    
    os.mkdir(os.path.join(png_root_folder + '/' + folder_machine), mode)
    os.mkdir(os.path.join(png_root_folder + '/' + folder_machine + '/png_train' ), mode)
    os.mkdir(os.path.join(png_root_folder + '/' + folder_machine + '/png_test' ), mode)
    for n in ['normal', 'anomaly']:
        os.mkdir(os.path.join(png_root_folder + '/' + folder_machine + '/png_train/' + n), mode)
        
    

In [3]:

# from wavs/*.wav, convert to pngs + split pngs in folders png_v3/normal + png_v3/anomaly (train data)
for folder_machine in utils.list_datasets: # ['valve']    
    for n in ['train', 'test']:
        dictStat = {'normal': 0, 'anomaly': 0} # init stats count of files in 2 classes
        use_folder = '../_data_origin/' + folder_machine + '/' + n + '/' 
        wavfiles = [f for f in listdir(use_folder) if isfile(join(use_folder, f))]
        if '.DS_Store' in wavfiles:
            wavfiles.remove('.DS_Store')
            
        nbWavs = len(wavfiles)
        countImages =  0    # counter for all images
        countImagesNormal =  0   
        countImagesAnomaly =  0
        # print('mmm:', folder_machine, n, nbWavs)

        for f in wavfiles:
            arrName = f.split("_") # anomaly_id_00_00000001.wav
            classPrefix = arrName[0] # 'normal' or 'anomaly'
            dictStat['normal'] =  dictStat['normal'] + 1 if classPrefix == 'normal' else dictStat['normal']
            dictStat['anomaly'] =  dictStat['anomaly'] + 1 if classPrefix == 'anomaly' else dictStat['anomaly']

        print('dictStat: ', dictStat) # 479 anomaly 3291 normal

        # random choose of images for testing at the end: put them in png_test folder_machine
        arrIndicesImagesToTestNormal = np.random.randint(1, dictStat['normal'], nbImagesTotestEachClass) if dictStat['anomaly'] > nbImagesTotestEachClass else []
        arrIndicesImagesToTestAnormal = np.random.randint(1, dictStat['anomaly'], nbImagesTotestEachClass) if dictStat['anomaly'] > nbImagesTotestEachClass else []
        print('nbWavs: ', nbWavs, ' in ', use_folder)
        print('test normal:', arrIndicesImagesToTestNormal)
        print('test anormal:', arrIndicesImagesToTestAnormal)

        for f in wavfiles:
            arrName = f.split("_") # anomaly_id_00_00000001.wav
            out_folder_png = ''
            classPrefix = arrName[0] # 'normal' or 'anomaly'
            countImages += 1
            if classPrefix == 'normal':
                countImagesNormal += 1
            if classPrefix == 'anomaly':    
                countImagesAnomaly += 1

            out_folder_png = '../_data_png/' + folder_machine + '/png_train/' + classPrefix + '/' # data/slider/png_v4/normal/
            out_folder_png_test = '../_data_png/' + folder_machine + '/png_test/'
            
            # use some anomaly for the training set:
            if classPrefix == 'normal' and countImagesNormal in arrIndicesImagesToTestNormal or classPrefix == 'anomaly' and countImagesAnomaly in arrIndicesImagesToTestAnormal:
                out_folder_png = out_folder_png_test
                # print('!!! image test: ', classPrefix)
                
            if countImages % 200 == 0:
                print('countImages generated...: ', countImages, ' / ', nbWavs, '(', folder_machine, use_folder + f, out_folder_png, ')')
            
            # print(use_folder + f, out_folder_png)
            s = SoundFile(use_folder + f, out_folder_png)
            s.exportMelSpectrogramColor() # create color file spectrogram 

print("*************** End ****************** duration: ",  datetime.datetime.now() - start_time) # 0:48:26


dictStat:  {'normal': 3291, 'anomaly': 0}
nbWavs:  3291  in  ../_data_origin/valve/train/
test normal: []
test anormal: []
countImages generated...:  200  /  3291 ( valve ../_data_origin/valve/train/normal_id_02_00000071.wav ../_data_png/valve/png_train/normal/ )
countImages generated...:  400  /  3291 ( valve ../_data_origin/valve/train/normal_id_04_00000840.wav ../_data_png/valve/png_train/normal/ )
countImages generated...:  600  /  3291 ( valve ../_data_origin/valve/train/normal_id_04_00000851.wav ../_data_png/valve/png_train/normal/ )
countImages generated...:  800  /  3291 ( valve ../_data_origin/valve/train/normal_id_00_00000494.wav ../_data_png/valve/png_train/normal/ )
countImages generated...:  1000  /  3291 ( valve ../_data_origin/valve/train/normal_id_04_00000388.wav ../_data_png/valve/png_train/normal/ )
countImages generated...:  1200  /  3291 ( valve ../_data_origin/valve/train/normal_id_00_00000432.wav ../_data_png/valve/png_train/normal/ )
countImages generated...:  14

*************** End ****************** duration:  0:48:26.629051
